- First download genienlp and checkout the master branch (skip this step if you're running this notebook within genienlp already)

In [ ]:
!git clone https://github.com/stanford-oval/genienlp.git genienlp; \
    cd genienlp ; \
    git checkout 096af96e3f61e7a3a0ffba630607463105e28eee ; \
    pip3 install -e .

- Now copy over your dataset to `datadir/almond/` folder. It should have at least train.tsv and eval.tsv files. If you want to predict on test set, you should include test.tsv file as well.
- Depending on the task the format of your data should be different. For a sample of data for almond task (semantic parsing) please checkout `tests/dataset/almond/` folder.

In [ ]:
# put your dataset in datadir/almond/
!mkdir -p datadir/almond/
!cp -r tests/dataset/almond/ datadir/almond/

- Follow instructions here https://github.com/HazyResearch/bootleg/tree/7ab94e794963409f5fe905becdd006c1588ababc
and run `download_bert.sh`, `download_model.sh`, and `download_wiki.sh`.
- Extract all the files into `database` directory
- Run the following command which will generate bootleg_label files in `results_temp` directory


In [ ]:
!genienlp bootleg-dump-features --train_tasks almond --save workdir/ --preserve_case --data datadir/  --database_type json --database_dir tests/database/ --ned_features type_id type_prob --ned_features_size 1 1 --ned_features_default_val 0 1.0 --num_workers 0 --min_entity_len 1 --max_entity_len 4 --bootleg_model bootleg_wiki_types --exist_ok


- Now you can use the extracted features from bootleg in downstream tasks such as semantic parsing
- Run following to train a model

In [ ]:
!genienlp train --train_tasks almond --train_iterations 60 --preserve_case --save workdir/ --data datadir/ --model TransformerSeq2Seq --pretrained_model facebook/bart-base --trainable_decoder_embeddings 50 --train_batch_tokens 1000 --val_batch_size 1000 --do_ned --database_type json --database_dir tests/database/ --ned_retrieve_method bootleg --ned_features type_id type_prob --ned_features_size 1 1 --ned_features_default_val 0 1.0 --num_workers 0 --min_entity_len 1 --max_entity_len 4 --bootleg_model bootleg_wiki_types --exist_ok --add_types_to_text append

- After training is done, you can evaluate the model using the following command.

In [ ]:
!genienlp predict --tasks almond --data datadir/ --path workdir/ --eval_dir eval_dir/ --evaluate test